In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta
from scipy.stats import norm
from scipy.optimize import minimize

In [11]:
def d(sigma, S, K, q, r, tau):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    return d1, d2

def call_price(sigma, S, K, q, r, tau):
    d1, d2 = d(sigma, S, K, q, r, tau)
    return S * np.exp(-q * tau) * norm.cdf(d1) - np.exp(-r * tau) * K * norm.cdf(d2)

def put_price(sigma, S, K, q, r, tau):
    d1, d2 = d(sigma, S, K, q, r, tau)
    return np.exp(-r * tau) * K * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1)

# 1st order greek
def call_delta(sigma, S, K, q, r, tau):
    d1, _ = d(sigma, S, K, q, r, tau)
    return np.exp(-q * tau) * norm.cdf(d1)

def put_delta(sigma, S, K, q, r, tau):
    d1, _ = d(sigma, S, K, q, r, tau)
    return -np.exp(-q * tau) * norm.cdf(-d1)

def vega(sigma, S, K, q, r, tau):
    d1, _ = d(sigma, S, K, q, r, tau)
    return S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)

def call_vega(sigma, S, K, q, r, tau):
    return vega(sigma, S, K, q, r, tau)

def put_vega(sigma, S, K, q, r, tau):
    return vega(sigma, S, K, q, r, tau)

def call_rho(sigma, S, K, q, r, tau):
    _, d2 = d(sigma, S, K, q, r, tau)
    return K * tau * np.exp(-r * tau) * norm.cdf(d2)

def put_rho(sigma, S, K, q, r, tau):
    _, d2 = d(sigma, S, K, q, r, tau)
    return -K * tau * np.exp(-r * tau) * norm.cdf(-d2)

def call_theta(sigma, S, K, q, r, tau):
    return -vega(sigma, S, K, q, r, tau) * sigma / (2 * tau) - r * call_rho(sigma, S, K, q, r, tau) / tau + q * S * call_delta(sigma, S, K, q, r, tau)

def put_theta(sigma, S, K, q, r, tau):
    return -vega(sigma, S, K, q, r, tau) * sigma / (2 * tau) - r * put_rho(sigma, S, K, q, r, tau) / tau + q * S * put_delta(sigma, S, K, q, r, tau)

def call_epsilon(sigma, S, K, q, r, tau):
    return -S * tau * call_delta(sigma, S, K, q, r, tau)

def put_epsilon(sigma, S, K, q, r, tau):
    return -S * tau * put_delta(sigma, S, K, q, r, tau)

def call_lambda(sigma, S, K, q, r, tau):
    return call_delta(sigma, S, K, q, r, tau) * S / call_price(sigma, S, K, q, r, tau)

def put_lambda(sigma, S, K, q, r, tau):
    return put_delta(sigma, S, K, q, r, tau) * S / put_price(sigma, S, K, q, r, tau)


# 2nd order greek
def gamma(sigma, S, K, q, r, tau):
    return vega(sigma, S, K, q, r, tau) / (S ** 2 * sigma * tau)

def vanna(sigma, S, K, q, r, tau):
    _, d2 = d(sigma, S, K, q, r, tau)
    return -vega(sigma, S, K, q, r, tau) * d2 / (S * sigma * np.sqrt(tau))

def vomma(sigma, S, K, q, r, tau):
    d1, d2 = d(sigma, S, K, q, r, tau)
    return vega(sigma, S, K, q, r, tau) * (d1 * d2 / sigma)

In [4]:
yesterday = datetime.today()-timedelta(days=1)
if yesterday.weekday() in [5, 6]:
    yesterday = yesterday - timedelta(datetime.today().weekday()) + timedelta(days=5)
yesterday = yesterday.replace(hour=21, minute=0, second=0, microsecond=0)

In [16]:
headers = {
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 OPR/84.0.4316.21"
}

symbol = "_SPX"
data = requests.get(f"https://cdn.cboe.com/api/global/delayed_quotes/options/{symbol}.json", headers=headers).json()["data"]
data

{'options': [{'option': 'SPX220318C00200000',
   'bid': 4112.0,
   'bid_size': 15.0,
   'ask': 4128.1,
   'ask_size': 15.0,
   'iv': 0.02,
   'open_interest': 506.0,
   'volume': 0.0,
   'delta': 0.9996,
   'gamma': 0.0,
   'theta': 0.0,
   'rho': 0.0821,
   'vega': 0.0006,
   'theo': 4118.519,
   'change': 0.0,
   'open': 0.0,
   'high': 0.0,
   'low': 0.0,
   'tick': 'up',
   'last_trade_price': 4162.88,
   'last_trade_time': '2022-03-03T16:10:02',
   'percent_change': 0.0,
   'prev_day_close': 4147.19995117188},
  {'option': 'SPX220318P00200000',
   'bid': 0.0,
   'bid_size': 0.0,
   'ask': 0.05,
   'ask_size': 30.0,
   'iv': 4.3583,
   'open_interest': 179.0,
   'volume': 0.0,
   'delta': 0.0,
   'gamma': 0.0,
   'theta': -0.0046,
   'rho': 0.0,
   'vega': 0.0004,
   'theo': 0.0241,
   'change': 0.0,
   'open': 0.0,
   'high': 0.0,
   'low': 0.0,
   'tick': 'no_change',
   'last_trade_price': 0.05,
   'last_trade_time': '2022-02-25T11:25:48',
   'percent_change': 0.0,
   'prev_day_

In [136]:
df = pd.DataFrame(data["options"]).set_index("option", drop=True)
df["underlying"] = data["current_price"]
df["expiry"] = pd.to_datetime(df.index.str.slice(-15,-9), format="%y%m%d") + pd.DateOffset(hours=21)
df["last_trade_time"] = pd.to_datetime(df.last_trade_time).dt.tz_localize(None)
df["flag"] = df.index.str.slice(-9, -8)
df["strike"] = df.index.str.slice(-8).astype(int) // 1000
df["tau"] = (df.expiry - yesterday) / timedelta(days=365)
df["moneyness"] = np.log(df.underlying / df.strike)
df["r"] = 0.0
df["q"] = 0.0
df["vanna"] = vanna(df.iv, df.underlying, df.strike, df.q, df.r, df.tau)
df["vomma"] = vomma(df.iv, df.underlying, df.strike, df.q, df.r, df.tau)
df

,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,gamma,theta,...,underlying,expiry,flag,strike,tau,moneyness,r,q,vanna,vomma
option,,,,,,,,,,,,,,,,,,,,,
SPX220318C00200000,4112.0,15.0,4128.10,15.0,0.0200,506.0,0.0,0.9996,0.0,0.0000,...,4328.8701,2022-03-18 21:00:00,C,200,0.038356,3.074744,0.0,0.0,-0.000000,0.000000
SPX220318P00200000,0.0,0.0,0.05,30.0,4.3583,179.0,0.0,0.0000,0.0,-0.0046,...,4328.8701,2022-03-18 21:00:00,P,200,0.038356,3.074744,0.0,0.0,-0.000087,0.296427
SPX220318C00400000,3912.1,15.0,3928.80,15.0,0.0200,36.0,0.0,0.9996,0.0,0.0000,...,4328.8701,2022-03-18 21:00:00,C,400,0.038356,2.381597,0.0,0.0,-0.000000,0.000000
SPX220318P00400000,0.0,0.0,0.05,2998.0,3.3760,39.0,42.0,0.0000,0.0,-0.0062,...,4328.8701,2022-03-18 21:00:00,P,400,0.038356,2.381597,0.0,0.0,-0.000169,0.564830
SPX220318C00600000,3712.1,15.0,3728.60,15.0,0.0200,3.0,0.0,0.9996,0.0,0.0000,...,4328.8701,2022-03-18 21:00:00,C,600,0.038356,1.976132,0.0,0.0,-0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SPXW221230P06600000,2246.3,1.0,2269.10,1.0,0.2351,2.0,1.0,-0.9946,0.0,0.0000,...,4328.8701,2022-12-30 21:00:00,P,6600,0.824658,-0.421763,0.0,0.0,0.616389,4528.145399
SPXW221230C06700000,0.0,0.0,4.10,300.0,0.1951,0.0,0.0,0.0044,0.0,-0.0137,...,4328.8701,2022-12-30 21:00:00,C,6700,0.824658,-0.436801,0.0,0.0,0.309844,2895.020190
SPXW221230P06700000,2345.1,1.0,2368.00,1.0,0.2391,0.0,0.0,-0.9957,0.0,0.0000,...,4328.8701,2022-12-30 21:00:00,P,6700,0.824658,-0.436801,0.0,0.0,0.578386,4327.161475


In [138]:
sel = df.loc[(df.tau == df.tau.min())]
sel = sel.loc[
    (sel.moneyness.abs() == sel.moneyness.abs().min()) |
    ((sel.moneyness + 0.025).abs() == (sel.moneyness + 0.025).abs().min()) |
    ((sel.moneyness - 0.025).abs() == (sel.moneyness - 0.025).abs().min())
]
sel

,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,gamma,theta,...,underlying,expiry,flag,strike,tau,moneyness,r,q,vanna,vomma
option,,,,,,,,,,,,,,,,,,,,,
SPXW220307C04220000,110.50,5.0,112.1,5.0,0.0200,23.0,15.0,0.8524,0.0023,-3.8388,...,4328.8701,2022-03-07 21:00:00,C,4220,0.008219,0.025471,0.0,0.0,-3.888414e-41,2.143870e-37
SPXW220307P04220000,7.90,22.0,8.3,21.0,0.2922,569.0,869.0,-0.1476,0.0022,-3.8419,...,4328.8701,2022-03-07 21:00:00,P,4220,0.008219,0.025471,0.0,0.0,-8.050777e-01,3.079834e+02
SPXW220307C04330000,30.10,1.0,30.6,1.0,0.1783,541.0,705.0,0.4789,0.0047,-6.2102,...,4328.8701,2022-03-07 21:00:00,C,4330,0.008219,-0.000261,0.0,0.0,5.420674e-02,1.715269e-01
SPXW220307P04330000,36.60,9.0,37.6,7.0,0.2740,146.0,642.0,-0.5210,0.0047,-6.2101,...,4328.8701,2022-03-07 21:00:00,P,4330,0.008219,-0.000261,0.0,0.0,3.338083e-02,-2.507708e-02
SPXW220307C04440000,1.25,21.0,1.4,27.0,0.1602,460.0,1460.0,0.0460,0.0014,-0.8683,...,4328.8701,2022-03-07 21:00:00,C,4440,0.008219,-0.025348,0.0,0.0,9.637854e-01,6.573885e+02
SPXW220307P04440000,115.50,38.0,121.1,38.0,0.3471,204.0,45.0,-0.9539,0.0014,-0.8648,...,4328.8701,2022-03-07 21:00:00,P,4440,0.008219,-0.025348,0.0,0.0,6.910082e-01,2.141789e+02


$$v = vanna, w = vomma$$
$$\frac{1}{2}\Gamma^{BS}S^2(\sigma^2-\Sigma^2)+\frac{1}{2}w^{BS}\Sigma^2\eta^2+v^{BS}S\Sigma\eta\sigma\rho = 0$$
$$\sigma = \pm \frac{(\sqrt{\Sigma^2 S^2 (\Gamma^2 S^2 + \rho^2 v^2 \eta^2 - \Gamma w\;\eta^2)}-\Sigma \rho\eta S v)}{\Gamma S^2}, \quad \Gamma\;S \neq 0$$
$$\eta = \pm \frac{(\sqrt{\Sigma^2 S^2 (\rho^2\sigma^2 v^2 + \Gamma w(\Sigma^2 - \sigma^2))}-\Sigma\rho\sigma S v)}{\Epsilon^2 w}, \quad \Sigma\;w \neq 0$$
$$\rho = \frac{\Gamma S^2(\Sigma^2-\sigma^2)-\Sigma^2\eta^2 w }{2 \Sigma\sigma\eta S v }, \quad \Sigma\sigma\eta S v \neq 0$$

In [161]:
def gvv(x, underlying, implied_volatility, gamma, vanna, vomma, debug=False):
    sigma, eta, rho = x
    S, epsilon = underlying, implied_volatility

    gamma_part = 0.5 * gamma * S ** 2 * (sigma ** 2 - epsilon ** 2)
    vanna_part = vanna * S * epsilon * eta * sigma * rho
    vomma_part = 0.5 * vomma * epsilon ** 2 * eta ** 2

    if debug: print(gamma_part + vanna_part + vomma_part)
    return np.mean((gamma_part + vanna_part + vomma_part) ** 2)

curr = sel
bounds = [(0, None), (0, None), (0, None)]
result = minimize(
    gvv, 
    [0, 1, 0], 
    args=(curr.underlying, curr.iv, curr.gamma, curr.vanna, curr.vomma),
    bounds=bounds,
    options={"maxiter": 1000}
)
result

      fun: 599585.1632849566
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.32830644e-02,  0.00000000e+00,  1.28481945e+05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 104
      nit: 11
     njev: 26
   status: 0
  success: True
        x: array([0.21737934, 7.0381353 , 0.        ])

In [162]:
gvv(result.x, curr.underlying, curr.iv, curr.gamma, curr.vanna, curr.vomma, debug=True)

option
SPXW220307C04220000    1009.698168
SPXW220307P04220000    -134.626437
SPXW220307C04330000     681.073096
SPXW220307P04330000   -1225.251665
SPXW220307C04440000     701.062345
SPXW220307P04440000    -321.412032
dtype: float64


599585.1632849566

In [18]:
atm_vegas  = vega(atm.impliedVolatility, atm.underlying, atm.strike, atm.q, atm.r, atm.tau)
atm_gammas = gamma(atm.impliedVolatility, atm.underlying, atm.strike, atm.q, atm.r, atm.tau)
atm_vannas = vanna(atm.impliedVolatility, atm.underlying, atm.strike, atm.q, atm.r, atm.tau)
atm_vommas = vomma(atm.impliedVolatility, atm.underlying, atm.strike, atm.q, atm.r, atm.tau)
wings_vegas  = vega(wings.impliedVolatility, wings.underlying, wings.strike, wings.q, wings.r, wings.tau)
wings_gammas = gamma(wings.impliedVolatility, wings.underlying, wings.strike, wings.q, wings.r, wings.tau)
wings_vannas = vanna(wings.impliedVolatility, wings.underlying, wings.strike, wings.q, wings.r, wings.tau)
wings_vommas = vomma(wings.impliedVolatility, wings.underlying, wings.strike, wings.q, wings.r, wings.tau)

In [74]:
call_price(wings.iloc[0].impliedVolatility, wings.iloc[0].underlying, wings.iloc[0].strike, wings.iloc[0].q, wings.iloc[0].r, wings.iloc[0].tau) - \
    call_price(atm.iloc[0].impliedVolatility, atm.iloc[0].underlying, atm.iloc[0].strike, atm.iloc[0].q, atm.iloc[0].r, atm.iloc[0].tau) - \
        put_price(wings.iloc[1].impliedVolatility, wings.iloc[1].underlying, wings.iloc[1].strike, wings.iloc[1].q, wings.iloc[1].r, wings.iloc[1].tau) + \
            put_price(atm.iloc[1].impliedVolatility, atm.iloc[1].underlying, atm.iloc[1].strike, atm.iloc[1].q, atm.iloc[1].r, atm.iloc[1].tau)

-0.11542062698329403

In [ ]:
def vanilla_vanna_cost(atm, wings):
    atm_vegas  = vega(atm.impliedVolatility, atm.underlying, atm.strike, atm.q, atm.r, atm.tau)
    atm_vannas = vanna(atm.impliedVolatility, atm.underlying, atm.strike, atm.q, atm.r, atm.tau)
    wings_vannas = vanna(wings.impliedVolatility, wings.underlying, wings.strike, wings.q, wings.r, wings.tau)

    B = (atm_vannas[0] - atm_vannas[1])
    return ()

def volga_cost(atm, wings):
    pass